# Merging the Economic and Light Data

### Reading in Data and Preparing to Merge

In [1]:
import pandas as pd
import pickle

econ_df = pd.read_csv('econ_data.csv')
df_lights = pickle.load(open('lights_data_prep.pkl','rb'))

# Renaming columns so that they are an exact match
df_lights.rename({'ADMIN':'Country', 'year':'Year'}, axis=1, inplace=True)

In [2]:
econ_df.head()

,Country,Year,Max_Partners,GDP_per_unit_CO2,PPP_Conv_Rate,PPP_Share_GDP,Imports_PC,Exports_PC,Govt_Revenue,gdp_per_cap,agri_perc_gdp,agg.empl.agri.perc,rural.pop.perc,pop.tot,mobilesub_per100peeps,intl_tourist_arrival,total_life_exp,life_expectancy_fe,life_exp_male,trade_perGDP
0,Albania,1990,75,2.504851,2.117,0.035,0.0,0.0,-6.424,1838.672777,36.410703,55.914001,63.572,3286542.0,0.0,1062000.0,71.836,74.991,69.070,39.436963
1,Albania,1991,75,2.684573,2.775,0.024,0.0,0.0,-6.424,1331.809147,36.410703,55.914001,63.300,3266790.0,0.0,1062000.0,71.803,74.980,69.017,36.070520
2,Albania,1992,75,4.443426,9.488,0.020,0.0,0.0,-6.424,1243.609404,36.410703,56.133999,62.751,3247039.0,0.0,1062000.0,71.802,74.985,68.997,108.785472
3,Albania,1993,75,5.264840,19.912,0.022,0.0,0.0,-6.424,1370.830016,36.410703,55.470001,62.201,3227287.0,0.0,1062000.0,71.860,75.039,69.037,80.518333
4,Albania,1994,75,5.542105,26.714,0.023,0.0,0.0,-6.424,1493.790201,36.410703,54.841000,61.646,3207536.0,0.0,1062000.0,71.992,75.158,69.150,53.102585


In [3]:
df_lights.head()

,Country,ISO_A3,cam,mean_light,Year,mean_light_diff,mean_light_pct_diff,mean_light_index
0,Afghanistan,AFG,F101993,0.027943,1993,NaN,NaN,100.000000
1,Afghanistan,AFG,F121994,0.018001,1994,-0.009942,-0.355793,64.420710
2,Afghanistan,AFG,F121995,0.026424,1995,0.008423,0.467920,94.564441
3,Afghanistan,AFG,F121996,0.029921,1996,0.003497,0.132342,107.079330
4,Afghanistan,AFG,F141997,0.026763,1997,-0.003159,-0.105566,95.775431


### Fixing Country Names

In [4]:
### Some of the country names in the two datasets do not match,
### so we need to fix them before merging.

# Country names that appear in Econ data
econ_countries = sorted(list(set(econ_df.iloc[:,0])))

# Country names that appear in OECD data
light_countries = sorted(list(set(df_lights.iloc[:,0])))

# Making csv for me to see what needs to be changed
Same = sorted(list(set(econ_countries).intersection(set(light_countries))))
econ_only = sorted(list(set(econ_countries) - set(light_countries)))
light_only = sorted(list(set(light_countries) - set(econ_countries)))
comparisondf = pd.DataFrame({'Same': pd.Series(Same),
                             'econ_only': pd.Series(econ_only),
                             'light_only': pd.Series(light_only)})
#comparisondf.to_csv('comp.csv')

In [5]:
# Changing names in df_lights to match econ_df
update_dic = {'Bolivia':'Bolivia, Plurinational State of',
              'Brunei':'Brunei Darussalam',
              'Czech Republic':'Czechia',
              'Ivory Coast':"Côte d'Ivoire",
              'South Korea':'Korea, Republic of',
              'Moldova':'Moldova, Republic of',
              'Macedonia':'North Macedonia',
              'Russia':'Russian Federation',
              'Slovak Republic':'Slovakia',
              'United Republic of Tanzania':'Tanzania, United Republic of',
              'United States of America':'United States',
              'Venezuela':'Venezuela, Bolivarian Republic of'}
df_lights = df_lights.replace({"Country": update_dic})
df_lights.sort_values(['Country', 'Year'], inplace=True)
df_lights.reset_index(drop=True, inplace=True)

### Merging

In [6]:
merged = pd.merge(left=econ_df, right=df_lights, how='inner', on=['Country', 'Year'])
merged.head()

,Country,Year,Max_Partners,GDP_per_unit_CO2,PPP_Conv_Rate,PPP_Share_GDP,Imports_PC,Exports_PC,Govt_Revenue,gdp_per_cap,...,total_life_exp,life_expectancy_fe,life_exp_male,trade_perGDP,ISO_A3,cam,mean_light,mean_light_diff,mean_light_pct_diff,mean_light_index
0,Albania,1993,75,5.264840,19.912,0.022,0.000000,0.000000,-6.424,1370.830016,...,71.860,75.039,69.037,80.518333,ALB,F101993,0.264269,NaN,NaN,100.000000
1,Albania,1994,75,5.542105,26.714,0.023,0.000000,0.000000,-6.424,1493.790201,...,71.992,75.158,69.150,53.102585,ALB,F121994,0.815907,0.551638,2.087411,308.741138
2,Albania,1995,75,6.905429,28.740,0.024,0.000000,0.000000,-6.424,1703.286747,...,72.205,75.352,69.347,47.610595,ALB,F121995,1.253462,0.437554,0.536279,474.312640
3,Albania,1996,75,7.477210,32.344,0.026,-4.943728,-9.512469,-6.424,1869.871255,...,72.495,75.618,69.631,44.895427,ALB,F121996,1.456427,0.202966,0.161924,551.115338
4,Albania,1997,72,8.689693,35.371,0.022,-9.901000,-19.051000,-6.424,1676.131932,...,72.838,75.931,69.982,45.425523,ALB,F141997,1.672326,0.215898,0.148238,632.811689


In [7]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2079 entries, 0 to 2078
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Country                2079 non-null   object 
 1   Year                   2079 non-null   int64  
 2   Max_Partners           2079 non-null   int64  
 3   GDP_per_unit_CO2       2079 non-null   float64
 4   PPP_Conv_Rate          2079 non-null   float64
 5   PPP_Share_GDP          2079 non-null   float64
 6   Imports_PC             2079 non-null   float64
 7   Exports_PC             2079 non-null   float64
 8   Govt_Revenue           2079 non-null   float64
 9   gdp_per_cap            2079 non-null   float64
 10  agri_perc_gdp          2079 non-null   float64
 11  agg.empl.agri.perc     2079 non-null   float64
 12  rural.pop.perc         2079 non-null   float64
 13  pop.tot                2079 non-null   float64
 14  mobilesub_per100peeps  2079 non-null   float64
 15  intl

### Writing to Pickle

In [8]:
merged.to_pickle('merged_data.pkl')

### Presentation Scatterplot

In [9]:
merged.gdp_per_cap.describe()

count      2079.000000
mean      16097.383031
std       19554.218630
min         178.801690
25%        2371.831862
50%        7067.620550
75%       25046.871462
max      111968.349501
Name: gdp_per_cap, dtype: float64

In [15]:
import plotly.express as px
import plotly
fig = px.scatter(merged[merged['Year'] == 1993], x = 'mean_light', y = 'gdp_per_cap', animation_group="Country")

fig.show()

In [12]:
fig = px.scatter(merged, x="mean_light", y="gdp_per_cap", animation_frame="Year", animation_group="Country",hover_name="Country",
           range_x=[0,70], range_y=[150,120000], title='Relationship between mean light and gdp per capita')
fig.show()

In [16]:
plotly.offline.plot(fig, filename='light_gdp.html')

'light_gdp.html'